# Beta Testing Opinions | Sentiment Analysis Model

_Author: Karolina Mamczarz_

_Based on: [Deep Learning Nanodegree Program | Udacity](https://www.udacity.com/course/deep-learning-nanodegree--nd101)_

## Description

PyTorch is used as a training tool. It is an open source machine learning framweork.

## Load dataset

Reaserch will use [Amazon Review Data (2018)](https://nijianmo.github.io/amazon/index.html), specifically data from [Software category](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Software_5.json.gz). Dataset downloaded on March 4th, 2020.

> _**Justifying recommendations using distantly-labeled reviews and fined-grained aspects**
Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019_